In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs, get_sum_cnt_words
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   385   0  0   1113   1103  1102  0.01  0.34  0.45  0.88  0.88
1   407   1  0   1091   1079  1077  0.02  0.38  0.45  0.91  0.91
2   407   2  0   1068   1055  1055  0.02  0.39  0.45  0.90  0.90
3   432   3  0   1046   1034  1034  0.02  0.41  0.45  0.92  0.92
4   458   4  0   1029   1019  1019  0.02  0.42  0.45  0.93  0.93
5   451   5  0   1015   1007  1006  0.03  0.42  0.45  0.92  0.92
6   454   6  0   1005    998   998  0.03  0.42  0.45  0.92  0.92
7   453   7  0    997    993   992  0.03  0.42  0.45  0.91  0.91
8   452   8  0    991    989   989  0.04  0.42  0.45  0.91  0.91
9   452   9  0    988    987   987  0.04  0.41  0.45  0.91  0.90
10  440  10  0    985    986   986  0.04  0.41  0.45  0.88  0.88
11  441  11  0    983    985   985  0.05  0.40  0.45  0.88  0.88
12  443  12  0    981    984   984  0.05  0.40  0.45  0.88  0.88
13  442  13  0    980    983   983  0.05  0.40  0.45  0.87  0.87
14  437  14  0    978    983   983  0.05  0.40  0.45  0.87  0.87
15  436  15  0    977    982   982  0.05  0.40  0.45  0.87  0.87
16  361  16  0    976    982   982  0.05  0.40  0.45  0.87  0.87
17  431  17  0    975    981   981  0.05  0.39  0.45  0.86  0.86
18  432  18  0    973    979   980  0.05  0.39  0.45  0.86  0.86
19  429  19  0    972    979   978  0.06  0.39  0.45  0.86  0.86
20  431  20  0    971    978   978  0.06  0.39  0.45  0.86  0.85
21  431  21  0    971    978   978  0.06  0.39  0.45  0.85  0.85
22  432  22  0    970    977   977  0.06  0.39  0.45  0.85  0.85
23  428  23  0    970    977   977  0.06  0.39  0.45  0.85  0.85
24  414  24  0    969    976   977  0.06  0.39  0.45  0.85  0.85
25  419  25  0    969    976   976  0.06  0.39  0.45  0.85  0.85
26  420  26  0    968    975   976  0.06  0.39  0.45  0.84  0.84
27  420  27  0    967    974   975  0.06  0.39  0.45  0.84  0.84
28  420  28  0    966    974   974  0.06  0.38  0.45  0.84  0.84
29  419  29  0    965    973   973  0.06  0.38  0.45  0.84  0.84
30  418  30  0    965    973   973  0.06  0.38  0.45  0.84  0.84
31  419  31  0    964    972   973  0.06  0.38  0.45  0.84  0.84
32  420  32  0    964    972   972  0.06  0.38  0.45  0.84  0.84
33  420  33  0    963    972   972  0.06  0.38  0.45  0.84  0.84
34  421  34  0    963    971   971  0.06  0.38  0.45  0.84  0.83
35  350  35  0    964    972   971  0.06  0.38  0.45  0.84  0.84
36  422  36  0    963    971   971  0.07  0.38  0.45  0.84  0.83
37  423  37  0    963    970   970  0.07  0.38  0.45  0.83  0.83
38  347  38  0    963    971   970  0.07  0.38  0.45  0.83  0.83
39  441  39  0    962    970   970  0.07  0.39  0.45  0.86  0.86
40  445  40  0    962    969   969  0.07  0.39  0.45  0.86  0.86
41  441  41  0    961    969   969  0.07  0.39  0.45  0.86  0.86
42  444  42  0    960    968   968  0.07  0.39  0.45  0.86  0.86
43  365  43  0    960    968   968  0.07  0.39  0.45  0.86  0.86
44  449  44  0    959    967   967  0.07  0.39  0.45  0.86  0.86
45  460  45  0    959    967   967  0.07  0.38  0.45  0.86  0.85
46  442  46  0    958    967   967  0.07  0.38  0.45  0.86  0.85
47  442  47  0    958    966   966  0.07  0.38  0.45  0.85  0.85
48  451  48  0    957    966   966  0.07  0.38  0.45  0.85  0.85
49  430  49  0    956    965   965  0.07  0.37  0.45  0.82  0.82

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta50-gam0.01-epoch50/log' mode='wb' closefd=True>


 0 11258 427997.0 use write get one article like know work also file
   0-1 7855 430884.0 say people one write think article know make go god
     0-1-1 7855 4058.0 professor ed kevin york patient san let university ac gordon
   0-2 1208 52352.0 game team play player win year season hockey go score
     0-2-1 1208 358.0 lose agree tell russia cpu germany mg font ad power
   0-3 74 4804.0 db output mov oname bh printf char int stream buf
     0-3-1 74 28.0 australia field controller longer cpu access main please compound cs
   0-4 1990 67228.0 key space use encryption information file chip technology system government
     0-4-1 1990 671.0 order thank usa research detector apr collection contact germany series
   0-5 131 3022.0 ground wire outlet wiring neutral circuit connect electrical panel nec
     0-5-1 131 91.0 ma mount accident dos ford root atlanta cpu channel happy
